# Pruebas para la generacion de datos de entrenamiento

## Cambios para que funcione con clases

Para exportar se ve que crear una clase para almacenar todas las percepciones es muy util por lo que se procede a modificar todas las funciones anteriores que lo requieran para que funcionen con la clase de percepciones.

#### Librerias

In [12]:
import os
import math
import pickle
from zmqRemoteApi import RemoteAPIClient
from random import randint as ri
from random import uniform as ru
import pandas as pd

#### Creacion de clases

In [29]:
#Class for simulation object
class sim_objects ():
    def __init__(self):
        self.client = RemoteAPIClient()
        self.sim = self.client.getObject('sim')
        self.file = 0
        self.executed = False
        self.joint_handler_ids = []
        self.num_joints = 0
        self.obj_handler_ids = []
        self.sequencies = 3
        self.seq_steps = 5
        self.training_data = []
        
sim_obj = sim_objects()

# Class to store the different handlers the robot needs to operate in the simulator. 
class joint_handlers_class ():
    def __init__(self):
        self.JRF0 = 0
        self.JRF1 = 0
        self.JRB0 = 0
        self.JRB1 = 0
        self.JLF0 = 0
        self.JLF1 = 0
        self.JLB0 = 0
        self.JLB1 = 0
        
handler = joint_handlers_class()

class obj_handlers_class ():
    def __init__(self):
        self.JRF2 = 0
        self.JRB2 = 0
        self.JLF2 = 0
        self.JLB2 = 0
        self.base = 0

obj_handler = obj_handlers_class()

#### Funciones de carga del quad y generacion de acciones random

In [52]:
def init_position():
    start_pos = ri(0,1)
    #Fixes the last joints position to 0 so they wont move
    for i in sim_obj.obj_handler_ids[:-1]:
        sim_obj.sim.setJointTargetPosition(i, 0)
    if start_pos == 0:
        #Set all the joints to their position
        for i in sim_obj.joint_handler_ids:
            sim_obj.sim.setJointTargetPosition(i, 0)
        #After setting all the joints it moves    
        for i in range(20):
            sim_obj.client.step()
    else:
        sim_obj.client.step()   
            

In [13]:
# Function that load the robot in the scene at the beggining of each evaluation.
def load_quad_class():
    # Opening the connection with CoppeliaSim simulator
    sim_obj.client = RemoteAPIClient()
    sim_obj.sim = sim_obj.client.getObject('sim')
    
    # Setting the simulation mode for working in sinchronous mode. The simulation timming is controlled by the Python program and not by the simulation itselfs.
    sim_obj.client.setStepping(True)
    
    #Right-front
    handler.JRF0 = sim_obj.sim.getObject("/JRF0")
    handler.JRF1 = sim_obj.sim.getObject("/JRF1")
    #Right-back
    handler.JRB0 = sim_obj.sim.getObject("/JRB0")
    handler.JRB1 = sim_obj.sim.getObject("/JRB1")
    #Left-front
    handler.JLF0 = sim_obj.sim.getObject("/JLF0")
    handler.JLF1 = sim_obj.sim.getObject("/JLF1")
    #Left-back
    handler.JLB0 = sim_obj.sim.getObject("/JLB0")
    handler.JLB1 = sim_obj.sim.getObject("/JLB1")

    #Setting obj handlers
    obj_handler.base = sim_obj.sim.getObject("/Base")
    obj_handler.JRF2 = sim_obj.sim.getObject("/JRF2")
    obj_handler.JRB2 = sim_obj.sim.getObject("/JRB2")
    obj_handler.JLF2 = sim_obj.sim.getObject("/JLF2")
    obj_handler.JLB2 = sim_obj.sim.getObject("/JLB2")

    
    #Stores the different id for every handler starting with the base
    sim_obj.joint_handler_ids = list(handler.__dict__.values())

    sim_obj.num_joints = len(sim_obj.joint_handler_ids)

    sim_obj.obj_handler_ids = list(obj_handler.__dict__.values())


    sim_obj.sim.startSimulation()

#### Clase y funcion para percepciones

In [14]:

#Class to store all the perceptions
class perceptions ():
    def __init__(self):
        #Execution data
        self.sequence = 0
        self.step = 0
        self.increments = []

        #Initial perceptions
        self.prev_j_positions = []
        self.prev_j_velocities = []
        self.prev_j_forces = []
        self.prev_base_pos_x = 0
        self.prev_base_pos_y = 0
        self.prev_base_pos_z = 0
        self.prev_base_ori_alpha = 0
        self.prev_base_ori_beta = 0
        self.prev_base_ori_gamma = 0
        #self.prev_base_accel = []

        #Resulting perceptions
        self.post_j_positions = []
        self.post_j_velocities = []
        self.post_j_forces = []
        self.post_base_pos_x = 0
        self.post_base_pos_y = 0
        self.post_base_pos_z = 0
        self.post_base_ori_alpha = 0
        self.post_base_ori_beta = 0
        self.post_base_ori_gamma = 0
        #self.post_base_accel = []

perception = perceptions()

#### Pruebas para guardar las percepciones por paso

In [15]:
def get_joints_preceptions():

    #Base perceptions
    base_x, base_y, base_z = sim_obj.sim.getObjectPosition(obj_handler.base, sim_obj.sim.handle_world)
    base_alpha, base_beta, base_gamma = sim_obj.sim.getObjectOrientation(obj_handler.base, sim_obj.sim.handle_world)
    if  not sim_obj.executed:
        perception.prev_base_pos_x = base_x
        perception.prev_base_pos_y = base_y
        perception.prev_base_pos_z = base_z
        perception.prev_base_ori_alpha = base_alpha
        perception.prev_base_ori_beta = base_beta
        perception.prev_base_ori_gamma = base_gamma
    else:
        perception.post_base_pos_x = base_x
        perception.post_base_pos_y = base_y
        perception.post_base_pos_z = base_z
        perception.post_base_ori_alpha = base_alpha
        perception.post_base_ori_beta = base_beta
        perception.post_base_ori_gamma = base_gamma
        
    #Joint perceptions
    for i in sim_obj.joint_handler_ids:
        #Angular/linear position of the joint
        pos = sim_obj.sim.getJointPosition(i)
        #Torque/force being aplied by the joint
        force = sim_obj.sim.getJointForce(i)
        #Angular/linear velocity of the joint's movement
        veloc = sim_obj.sim.getJointVelocity(i)
        if  not sim_obj.executed:
            perception.prev_j_positions.append(pos)
            perception.prev_j_velocities.append(veloc)
            perception.prev_j_forces.append(force)
        else:
            perception.post_j_positions.append(pos)
            perception.post_j_velocities.append(veloc)
            perception.post_j_forces.append(force)

In [16]:
def move_joints():
    for i in range(sim_obj.num_joints):
        sim_obj.sim.setJointTargetPosition(sim_obj.joint_handler_ids[i], (perception.increments[i]+perception.prev_j_positions[i]))
    #After setting all the joints it moves
    for i in range(5):
        sim_obj.client.step()

In [17]:
#Generate list of random positions
def rand_gen():
    for i in range(sim_obj.num_joints):
        n = ru(-10, 10) * math.pi / 180
        perception.increments.append(n)

#### Exportar

In [65]:
def export_pickle():    
    #Dump data in file
    sim_obj.file = open(os.getcwd() + r'\training_dataset.txt', 'wb')
    pickle.dump(sim_obj.training_data, sim_obj.file)
    sim_obj.file.close()

#### Ejecucion

##### Multiples secuencias y multiples pasos

##### Pruebas finales con el main

In [85]:
sim_obj.sequencies = 4
print("Numero de secuencias: ",sim_obj.sequencies)
sim_obj.seq_steps = 3
print("Steps por secuencia: ",sim_obj.seq_steps)
sim_obj.training_data = []

Numero de secuencias:  4
Steps por secuencia:  3


In [90]:
#Main function for multiple executions and multiple steps
#def main():
print("main function")
sim_obj.training_data = []
for k in range(sim_obj.sequencies):
    load_quad_class()
    init_position()
    for j in range(sim_obj.seq_steps):
        perception = perceptions()
        perception.sequence = k
        perception.step = j         
        print("----------- Secuencia: ", perception.sequence)
        print("----------- Paso: ", perception.step)
        rand_gen()
        get_joints_preceptions()
        #Execute every joint action
        move_joints()
        sim_obj.executed = True
        get_joints_preceptions()
        sim_obj.executed = False
        sim_obj.training_data.append(vars(perception))
    sim_obj.sim.stopSimulation()
#export()

main function
Flat
----------- Secuencia:  0
----------- Paso:  0
----------- Secuencia:  0
----------- Paso:  1
----------- Secuencia:  0
----------- Paso:  2
Flat
----------- Secuencia:  1
----------- Paso:  0
----------- Secuencia:  1
----------- Paso:  1
----------- Secuencia:  1
----------- Paso:  2
Standing
----------- Secuencia:  2
----------- Paso:  0
----------- Secuencia:  2
----------- Paso:  1
----------- Secuencia:  2
----------- Paso:  2
Flat
----------- Secuencia:  3
----------- Paso:  0
----------- Secuencia:  3
----------- Paso:  1
----------- Secuencia:  3
----------- Paso:  2


##### Pruebas con pandas

In [91]:
df = pd.DataFrame(sim_obj.training_data)
df

,sequence,step,increments,prev_j_positions,prev_j_velocities,prev_j_forces,prev_base_pos_x,prev_base_pos_y,prev_base_pos_z,prev_base_ori_alpha,...,prev_base_ori_gamma,post_j_positions,post_j_velocities,post_j_forces,post_base_pos_x,post_base_pos_y,post_base_pos_z,post_base_ori_alpha,post_base_ori_beta,post_base_ori_gamma
0,0,0,"[0.12649222499953922, -0.02283292362893596, -0...","[-0.0010654926300048828, -0.04230475425720215,...","[-0.0061512053944170475, 0.0385761633515358, 0...","[-0.018592823296785355, 0.15814179182052612, 0...",0.000022,-0.000774,0.033502,0.000053,...,0.002250,"[0.12603044509887695, -0.10594701766967773, -0...","[0.035476718097925186, 0.11401187628507614, 0....","[-0.0008254386484622955, 0.12692295014858246, ...",0.002179,-0.001341,0.036847,0.003460,0.005102,0.025929
1,0,1,"[-0.117137965169735, -0.00830241518342736, -0....","[0.12603044509887695, -0.10594701766967773, -0...","[0.035476718097925186, 0.11401187628507614, 0....","[-0.0008254386484622955, 0.12692295014858246, ...",0.002179,-0.001341,0.036847,0.003460,...,0.025929,"[0.010022163391113281, -0.1569533348083496, -0...","[-0.009346017614006996, 0.05078320577740669, -...","[-0.03095586597919464, 0.19419336318969727, 0....",0.008719,-0.000843,0.036980,-0.003013,0.011240,0.019618
2,0,2,"[-0.0932235364555353, 0.15385486808366947, 0.1...","[0.010022163391113281, -0.1569533348083496, -0...","[-0.009346017614006996, 0.05078320577740669, -...","[-0.03095586597919464, 0.19419336318969727, 0....",0.008719,-0.000843,0.036980,-0.003013,...,0.019618,"[-0.08218502998352051, -0.04079580307006836, -...","[-0.01649858057498932, 0.2299787849187851, 0.0...","[0.023740509524941444, 0.13115733861923218, -0...",0.007281,0.002127,0.038514,-0.005605,0.052504,-0.005313
3,1,0,"[-0.06863673839126473, 0.08828472154303071, 0....","[-0.0010654926300048828, -0.04230475425720215,...","[-0.0061512053944170475, 0.0385761633515358, 0...","[-0.018592823296785355, 0.15814179182052612, 0...",0.000022,-0.000774,0.033502,0.000053,...,0.002250,"[-0.06345129013061523, 0.005017757415771484, 0...","[-0.17719285190105438, 0.23202918469905853, 0....","[0.1143677681684494, 0.11438461393117905, -0.1...",-0.003249,-0.003061,0.035764,0.004331,-0.001884,-0.009390
4,1,1,"[0.14790019061772586, -0.08850513981401781, -0...","[-0.06345129013061523, 0.005017757415771484, 0...","[-0.17719285190105438, 0.23202918469905853, 0....","[0.1143677681684494, 0.11438461393117905, -0.1...",-0.003249,-0.003061,0.035764,0.004331,...,-0.009390,"[0.07783985137939453, -0.11231708526611328, 0....","[0.3846172094345093, -0.1111508458852768, -0.1...","[-0.19164562225341797, 0.46284347772598267, 0....",-0.003244,-0.006890,0.037363,0.013865,-0.011680,-0.030703
5,1,2,"[-0.06253357761674555, -0.06083315250911609, 0...","[0.07783985137939453, -0.11231708526611328, 0....","[0.3846172094345093, -0.1111508458852768, -0.1...","[-0.19164562225341797, 0.46284347772598267, 0....",-0.003244,-0.006890,0.037363,0.013865,...,-0.030703,"[0.013831138610839844, -0.20002365112304688, 0...","[-0.011253368109464645, 0.009632119908928871, ...","[-0.03591739386320114, 0.5402714014053345, -0....",-0.005871,-0.008007,0.039180,0.013015,-0.002345,0.003956
6,2,0,"[-0.1293783406743649, 0.045238925987848434, -0...","[-0.0004067420959472656, 1.5587503910064697, 0...","[-0.46758660674095154, 0.39749154448509216, 0....","[-0.013305321335792542, 0.05467931181192398, -...",0.000006,-0.000882,0.117879,0.000037,...,-0.000120,"[-0.12984609603881836, 1.573868989944458, -0.0...","[-0.008964546956121922, 0.6238943338394165, -0...","[0.0007454557344317436, 1.1438368558883667, 0....",-0.000281,0.008243,0.118653,-0.003079,0.004715,0.034661
7,2,1,"[-0.11626931913177013, -0.1707370504235278, 0....","[-0.12984609603881836, 1.573868989944458, -0.0...","[-0.008964546956121922, 0.6238943338394165, -0...","[0.0007454557344317436, 1.1438368558883667, 0....",-0.000281,0.008243,0.118653,-0.003079,...,0.034661,"[-0.24599099159240723, 1.413752794265747, -0.0...","[-0.1076222509145

In [31]:
df.to_csv("data_ipynb.csv", index=False)

#### Pruebas para toma de datos unica

In [56]:
import os
import math
import pickle
from zmqRemoteApi import RemoteAPIClient
from random import randint as ri
from random import uniform as ru
import pandas as pd

In [73]:
#Class for simulation object
class sim_objects ():
    def __init__(self):
        self.client = 0
        self.sim = 0
        self.file = 0
        self.joint_handler_ids = []
        self.num_joints = 0
        self.obj_handler_ids = []
        self.sequencies = 2
        self.seq_steps = 3
        self.training_data = []
        
        #Stores initial perceptions measured in the load function
        self.initial_j_positions = []
        self.initial_j_velocities = []
        self.initial_j_forces = []
        self.initial_base_pos_x = 0
        self.initial_base_pos_y = 0
        self.initial_base_pos_z = 0
        self.initial_base_ori_alpha = 0
        self.initial_base_ori_beta = 0
        self.initial_base_ori_gamma = 0
        
sim_obj = sim_objects()

# Class to store the different handlers the robot needs to operate in the simulator. 
class joint_handlers_class ():
    def __init__(self):
        self.JRF0 = 0
        self.JRF1 = 0
        self.JRB0 = 0
        self.JRB1 = 0
        self.JLF0 = 0
        self.JLF1 = 0
        self.JLB0 = 0
        self.JLB1 = 0
        
handler = joint_handlers_class()

class obj_handlers_class ():
    def __init__(self):
        self.JRF2 = 0
        self.JRB2 = 0
        self.JLF2 = 0
        self.JLB2 = 0
        self.base = 0

obj_handler = obj_handlers_class()

In [74]:
# Function that load the robot in the scene at the beggining of each evaluation.
def load_quad_class():
    # Opening the connection with CoppeliaSim simulator    
    sim_obj.client = RemoteAPIClient()
    sim_obj.sim = sim_obj.client.getObject('sim')
    
    # Setting the simulation mode for working in sinchronous mode. The simulation timming is controlled by the Python program and not by the simulation itselfs.
    sim_obj.client.setStepping(True)
    
    #Right-front
    handler.JRF0 = sim_obj.sim.getObject("/JRF0")
    handler.JRF1 = sim_obj.sim.getObject("/JRF1")
    #Right-back
    handler.JRB0 = sim_obj.sim.getObject("/JRB0")
    handler.JRB1 = sim_obj.sim.getObject("/JRB1")
    #Left-front
    handler.JLF0 = sim_obj.sim.getObject("/JLF0")
    handler.JLF1 = sim_obj.sim.getObject("/JLF1")
    #Left-back
    handler.JLB0 = sim_obj.sim.getObject("/JLB0")
    handler.JLB1 = sim_obj.sim.getObject("/JLB1")

    #Setting obj handlers
    obj_handler.base = sim_obj.sim.getObject("/Base")
    obj_handler.JRF2 = sim_obj.sim.getObject("/JRF2")
    obj_handler.JRB2 = sim_obj.sim.getObject("/JRB2")
    obj_handler.JLF2 = sim_obj.sim.getObject("/JLF2")
    obj_handler.JLB2 = sim_obj.sim.getObject("/JLB2")

    
    #Stores the different id for every handler starting with the base
    sim_obj.joint_handler_ids = list(handler.__dict__.values())

    sim_obj.num_joints = len(sim_obj.joint_handler_ids)

    sim_obj.obj_handler_ids = list(obj_handler.__dict__.values())

    sim_obj.sim.startSimulation()

In [75]:
def init_position():
    start_pos = ri(0,1)
    #Fixes the last joints position to 0 so they wont move
    for i in sim_obj.obj_handler_ids[:-1]:
        sim_obj.sim.setJointTargetPosition(i, 0)
    if start_pos == 0:
        #Set all the joints to their position
        for i in sim_obj.joint_handler_ids:
            sim_obj.sim.setJointTargetPosition(i, 0)
        #After setting all the joints it moves    
        for i in range(20):
            sim_obj.client.step()
    else:
        sim_obj.client.step()
    
    #Starting initial perceptions
    sim_obj.initial_base_pos_x = 0
    sim_obj.initial_base_pos_y = 0
    sim_obj.initial_base_pos_z = 0
    sim_obj.initial_base_ori_alpha = 0
    sim_obj.initial_base_ori_beta = 0
    sim_obj.initial_base_ori_gamma = 0
    sim_obj.initial_j_positions = []
    sim_obj.initial_j_velocities = []
    sim_obj.initial_j_forces = []

    #Get initial perceptions
    base_x, base_y, base_z = sim_obj.sim.getObjectPosition(obj_handler.base, sim_obj.sim.handle_world)
    base_alpha, base_beta, base_gamma = sim_obj.sim.getObjectOrientation(obj_handler.base, sim_obj.sim.handle_world)
    sim_obj.initial_base_pos_x = base_x
    sim_obj.initial_base_pos_x = base_y
    sim_obj.initial_base_pos_x = base_z
    sim_obj.initial_base_ori_alpha = base_alpha
    sim_obj.initial_base_ori_beta = base_beta
    sim_obj.initial_base_ori_gamma = base_gamma
        
    #Joint perceptions
    for i in sim_obj.joint_handler_ids:
        #Angular/linear position of the joint
        pos = sim_obj.sim.getJointPosition(i)
        #Torque/force being aplied by the joint
        force = sim_obj.sim.getJointForce(i)
        #Angular/linear velocity of the joint's movement
        veloc = sim_obj.sim.getJointVelocity(i)
        sim_obj.initial_j_positions.append(pos)
        sim_obj.initial_j_velocities.append(veloc)
        sim_obj.initial_j_forces.append(force)

In [76]:
def move_joints():
    if perception.step != 0:
        for i in range(sim_obj.num_joints):
            prev_pos = sim_obj.training_data[-1]['post_j_positions']
            sim_obj.sim.setJointTargetPosition(sim_obj.joint_handler_ids[i], (perception.increments[i]+prev_pos[i]))
        #After setting all the joints it moves
        for i in range(5):
            sim_obj.client.step()
    else:
        for i in range(sim_obj.num_joints):
            sim_obj.sim.setJointTargetPosition(sim_obj.joint_handler_ids[i], (perception.increments[i]+sim_obj.initial_j_positions[i]))
        #After setting all the joints it moves
        for i in range(5):
            sim_obj.client.step()

#Generate list of random positions
def rand_gen():
    for i in range(sim_obj.num_joints):
        n = ru(-7.5, 7.5) * math.pi / 180
        perception.increments.append(n)

In [77]:
#Class to store all the perceptions
class perceptions ():
    def __init__(self):
        #Execution data
        self.sequence = 0
        self.step = 0
        self.increments = []

        #Initial perceptions
        self.prev_j_positions = []
        self.prev_j_velocities = []
        self.prev_j_forces = []
        self.prev_base_pos_x = 0
        self.prev_base_pos_y = 0
        self.prev_base_pos_z = 0
        self.prev_base_ori_alpha = 0
        self.prev_base_ori_beta = 0
        self.prev_base_ori_gamma = 0
        #self.prev_base_accel = []

        #Resulting perceptions
        self.post_j_positions = []
        self.post_j_velocities = []
        self.post_j_forces = []
        self.post_base_pos_x = 0
        self.post_base_pos_y = 0
        self.post_base_pos_z = 0
        self.post_base_ori_alpha = 0
        self.post_base_ori_beta = 0
        self.post_base_ori_gamma = 0
        #self.post_base_accel = []

perception = perceptions()

In [78]:
def get_preceptions():
    print(perception.step)
    #Define previous perceptions
    if perception.step != 0:
        perception.prev_base_pos_x = sim_obj.training_data[-1]['post_base_pos_x']
        perception.prev_base_pos_y = sim_obj.training_data[-1]['post_base_pos_y']
        perception.prev_base_pos_z = sim_obj.training_data[-1]['post_base_pos_z']
        perception.prev_base_ori_alpha = sim_obj.training_data[-1]['post_base_ori_alpha']
        perception.prev_base_ori_beta = sim_obj.training_data[-1]['post_base_ori_beta']
        perception.prev_base_ori_gamma = sim_obj.training_data[-1]['post_base_ori_gamma']
        perception.prev_j_positions = sim_obj.training_data[-1]['post_j_positions']
        perception.prev_j_velocities = sim_obj.training_data[-1]['post_j_velocities']
        perception.prev_j_forces = sim_obj.training_data[-1]['post_j_forces']    
    else:
        print("Primer step")
        perception.prev_base_pos_x = sim_obj.initial_base_pos_x 
        perception.prev_base_pos_y = sim_obj.initial_base_pos_y
        perception.prev_base_pos_z = sim_obj.initial_base_pos_z
        perception.prev_base_ori_alpha = sim_obj.initial_base_ori_alpha
        perception.prev_base_ori_beta = sim_obj.initial_base_ori_beta
        perception.prev_base_ori_gamma = sim_obj.initial_base_ori_gamma
        perception.prev_j_positions = sim_obj.initial_j_positions
        perception.prev_j_velocities = sim_obj.initial_j_velocities
        perception.prev_j_forces = sim_obj.initial_j_forces
        

    #Get perceptions
    base_x, base_y, base_z = sim_obj.sim.getObjectPosition(obj_handler.base, sim_obj.sim.handle_world)
    base_alpha, base_beta, base_gamma = sim_obj.sim.getObjectOrientation(obj_handler.base, sim_obj.sim.handle_world)
    perception.post_base_pos_x = base_x
    perception.post_base_pos_y = base_y
    perception.post_base_pos_z = base_z
    perception.post_base_ori_alpha = base_alpha
    perception.post_base_ori_beta = base_beta
    perception.post_base_ori_gamma = base_gamma
        
    #Joint perceptions
    for i in sim_obj.joint_handler_ids:
        #Angular/linear position of the joint
        pos = sim_obj.sim.getJointPosition(i)
        #Torque/force being aplied by the joint
        force = sim_obj.sim.getJointForce(i)
        #Angular/linear velocity of the joint's movement
        veloc = sim_obj.sim.getJointVelocity(i)
        perception.post_j_positions.append(pos)
        perception.post_j_velocities.append(veloc)
        perception.post_j_forces.append(force)

In [79]:
sim_obj.sequencies = 2
print("Numero de secuencias: ",sim_obj.sequencies)
sim_obj.seq_steps = 4
print("Steps por secuencia: ",sim_obj.seq_steps)
perception = perceptions()

Numero de secuencias:  2
Steps por secuencia:  4


In [80]:
#Main function for multiple executions and multiple steps
#def main():
print("main function")
sim_obj = sim_objects()
perception = perceptions()
for k in range(sim_obj.sequencies):
    load_quad_class()
    init_position()
    for j in range(sim_obj.seq_steps):
        perception = perceptions()
        perception.sequence = k
        perception.step = j    
        print(f"----------- Secuencia: {perception.sequence} Paso: {perception.step}")
        rand_gen()
        #Execute every joint action
        move_joints()
        get_preceptions()

        sim_obj.training_data.append(vars(perception))
        print(sim_obj.training_data[0]["prev_j_positions"])
        print(sim_obj.training_data[-1]["prev_j_positions"])
        #Problema es que la posicion inicial de cada secuencia la guarda siempre en el prev percept de la primera secuencia

    sim_obj.sim.stopSimulation()

main function


----------- Secuencia: 0 Paso: 0
0
Primer step
[0.004590511322021484, 1.5506393909454346, 0.0034737586975097656, 1.5520603656768799, -0.0009093284606933594, 1.54378342628479, -0.0015537738800048828, 1.5506432056427002]
[0.004590511322021484, 1.5506393909454346, 0.0034737586975097656, 1.5520603656768799, -0.0009093284606933594, 1.54378342628479, -0.0015537738800048828, 1.5506432056427002]
----------- Secuencia: 0 Paso: 1
1
[0.004590511322021484, 1.5506393909454346, 0.0034737586975097656, 1.5520603656768799, -0.0009093284606933594, 1.54378342628479, -0.0015537738800048828, 1.5506432056427002]
[-0.005868196487426758, 1.5714857578277588, 0.032085418701171875, 1.4823191165924072, 0.0664825439453125, 1.571087121963501, 0.009787559509277344, 1.564164400100708]
----------- Secuencia: 0 Paso: 2
2
[0.004590511322021484, 1.5506393909454346, 0.0034737586975097656, 1.5520603656768799, -0.0009093284606933594, 1.54378342628479, -0.0015537738800048828, 1.5506432056427002]
[-0.131089448928833, 1.571138

In [81]:
print(len(sim_obj.training_data[0]["prev_j_positions"]))

8


In [82]:
# La posicion final del paso anterior es la posicion inicial del paso siguiente
if sim_obj.training_data[-1]['prev_j_positions'] == sim_obj.training_data[-2]['post_j_positions']:
    print("Iguales")

Iguales


In [83]:
df = pd.DataFrame(sim_obj.training_data)
df

,sequence,step,increments,prev_j_positions,prev_j_velocities,prev_j_forces,prev_base_pos_x,prev_base_pos_y,prev_base_pos_z,prev_base_ori_alpha,...,prev_base_ori_gamma,post_j_positions,post_j_velocities,post_j_forces,post_base_pos_x,post_base_pos_y,post_base_pos_z,post_base_ori_alpha,post_base_ori_beta,post_base_ori_gamma
0,0,0,"[-0.010995643197727814, 0.08048864777288355, 0...","[0.004590511322021484, 1.5506393909454346, 0.0...","[1.8352035284042358, -0.3187180161476135, -0.0...","[-0.005172162316739559, 0.008375206962227821, ...",0.117928,0.000000,0.000000,0.000101,...,-0.000087,"[-0.005868196487426758, 1.5714857578277588, 0....","[-0.05221371725201607, -0.05626683682203293, -...","[-0.033352501690387726, 1.1115632057189941, 0....",0.001736,0.005483,0.118523,-0.001878,0.007587,-0.019580
1,0,1,"[-0.13061083796019102, 0.08692298104478735, -0...","[-0.005868196487426758, 1.5714857578277588, 0....","[-0.05221371725201607, -0.05626683682203293, -...","[-0.033352501690387726, 1.1115632057189941, 0....",0.001736,0.005483,0.118523,-0.001878,...,-0.019580,"[-0.131089448928833, 1.571138620376587, 0.0188...","[-0.27799633145332336, -0.2418520301580429, -0...","[0.15436266362667084, 1.5161325931549072, -0.0...",0.003172,-0.000207,0.118820,0.001103,0.009582,-0.015675
2,0,2,"[-0.11675972921903754, -0.06597575731673973, -...","[-0.131089448928833, 1.571138620376587, 0.0188...","[-0.27799633145332336, -0.2418520301580429, -0...","[0.15436266362667084, 1.5161325931549072, -0.0...",0.003172,-0.000207,0.118820,0.001103,...,-0.015675,"[-0.24663352966308594, 1.5166895389556885, -0....","[0.020694753155112267, -0.3089907765388489, -0...","[0.07136896997690201, 0.28491976857185364, -0....",0.003352,-0.002537,0.118780,-0.000510,-0.005584,0.012285
3,1,0,"[-0.09297420733969043, -0.11761048342212609, -...","[-0.0035386085510253906, 1.56325364112854, -0....","[0.33674272894859314, -0.3040316700935364, 0.6...","[0.004353030119091272, -0.13012182712554932, 0...",0.117828,0.000000,0.000000,-0.000175,...,-0.000085,"[-0.10129022598266602, 1.4537298679351807, -0....","[-0.051163457334041595, 0.6984550952911377, -0...","[-0.08699142932891846, 0.7685973048210144, 0.0...",0.007579,0.004822,0.119613,0.002132,-0.005780,-0.005536
4,1,1,"[-0.06274547781507847, 0.10811975485657511, -0...","[-0.10129022598266602, 1.4537298679351807, -0....","[-0.051163457334041595, 0.6984550952911377, -0...","[-0.08699142932891846, 0.7685973048210144, 0.0...",0.007579,0.004822,0.119613,0.002132,...,-0.005536,"[-0.1681041717529297, 1.563020944595337, -0.18...","[-0.14795918762683868, 0.055693306028842926, -...","[-0.11047603189945221, 0.19348600506782532, 0....",0.011391,-0.005506,0.119166,0.005856,-0.003653,-0.004892
5,1,2,"[-0.050471566303370255, -0.02037907093982686, ...","[-0.1681041717529297, 1.563020944595337, -0.18...","[-0.14795918762683868, 0.055693306028842926, -...","[-0.11047603189945221, 0.19348600506782532, 0....",0.011391,-0.005506,0.119166,0.005856,...,-0.004892,"[-0.21924662590026855, 1.5497791767120361, -0....","[0.013732596300542355, -0.029181765392422676, ...","[-0.02951255440711975, 0.28611281514167786, -0...",0.014342,-0.007356,0.119694,0.007077,0.000655,-0.001032


In [84]:
df.to_csv("data_ipynb.csv", index=False)

In [85]:
sim_obj.file = open('pickled_training_dataset_ipynb.txt', 'wb')
pickle.dump(sim_obj.training_data, sim_obj.file)
sim_obj.file.close()

# Pruebas para funcion main

In [12]:
import os, sys, math, pickle, time
from zmqRemoteApi import RemoteAPIClient
from random import randint as ri
from random import uniform as ru
import pandas as pd

cluster = False
if cluster == True:
    port_conexion = int(sys.argv[1])
    print ('Base Port:', port_conexion)
# Whether the CESGA is not used, select the port '23000' by default.
else:
    port_conexion = 23000

#Class for simulation object
class sim_objects ():
    def __init__(self):
        self.client = 0
        self.sim = 0
        self.file = 0
        self.joint_handler_ids = []
        self.num_joints = 0
        self.obj_handler_ids = []
        
        #Number of sequences and steps per sequence
        self.sequencies = 2
        self.seq_steps = 4
        #self.sequencies = 1000
        #self.seq_steps = 10

        #Stores all the training data to be saved
        self.training_data = []

        #Stores initial perceptions measured in the load function
        self.initial_j_positions = []
        self.initial_j_velocities = []
        self.initial_j_forces = []
        self.initial_base_pos_x = 0
        self.initial_base_pos_y = 0
        self.initial_base_pos_z = 0
        self.initial_base_ori_alpha = 0
        self.initial_base_ori_beta = 0
        self.initial_base_ori_gamma = 0
        
#sim_obj = sim_objects()


# Class to store the different handlers the robot needs to operate in the simulator. 
class joint_handlers_class ():
    def __init__(self):
        self.JRF0 = 0
        self.JRF1 = 0
        self.JRB0 = 0
        self.JRB1 = 0
        self.JLF0 = 0
        self.JLF1 = 0
        self.JLB0 = 0
        self.JLB1 = 0
        
#handler = joint_handlers_class()

class obj_handlers_class ():
    def __init__(self):
        self.JRF2 = 0
        self.JRB2 = 0
        self.JLF2 = 0
        self.JLB2 = 0
        self.base = 0

#obj_handler = obj_handlers_class()


#Class to store all the perceptions
class perceptions ():
    def __init__(self):
        #Execution data
        self.sequence = 0
        self.step = 0
        self.increments = []
        #Initial perceptions
        self.prev_j_positions = []
        self.prev_j_velocities = []
        self.prev_j_forces = []
        self.prev_base_pos_x = 0
        self.prev_base_pos_y = 0
        self.prev_base_pos_z = 0
        self.prev_base_ori_alpha = 0
        self.prev_base_ori_beta = 0
        self.prev_base_ori_gamma = 0
        #Resulting perceptions
        self.post_j_positions = []
        self.post_j_velocities = []
        self.post_j_forces = []
        self.post_base_pos_x = 0
        self.post_base_pos_y = 0
        self.post_base_pos_z = 0
        self.post_base_ori_alpha = 0
        self.post_base_ori_beta = 0
        self.post_base_ori_gamma = 0

#perception = perceptions()

In [13]:
def init_classes():
    sim_obj = sim_objects()
    handler = joint_handlers_class()
    obj_handler = obj_handlers_class()
    perception = perceptions()

# Function that load the robot in the scene at the beggining of each evaluation.
def load_quad_class():
    # Opening the connection with CoppeliaSim simulator
    sim_obj.client = RemoteAPIClient('localhost', port_conexion)
    sim_obj.sim = sim_obj.client.getObject('sim')
    # Setting the simulation mode for working in sinchronous mode. The simulation timming is controlled by the Python program and not by the simulation itselfs.
    sim_obj.client.setStepping(True)
    #Right-front
    handler.JRF0 = sim_obj.sim.getObject("/JRF0")
    handler.JRF1 = sim_obj.sim.getObject("/JRF1")
    #Right-back
    handler.JRB0 = sim_obj.sim.getObject("/JRB0")
    handler.JRB1 = sim_obj.sim.getObject("/JRB1")
    #Left-front
    handler.JLF0 = sim_obj.sim.getObject("/JLF0")
    handler.JLF1 = sim_obj.sim.getObject("/JLF1")
    #Left-back
    handler.JLB0 = sim_obj.sim.getObject("/JLB0")
    handler.JLB1 = sim_obj.sim.getObject("/JLB1")
    #Setting obj handlers
    obj_handler.base = sim_obj.sim.getObject("/Base")
    obj_handler.JRF2 = sim_obj.sim.getObject("/JRF2")
    obj_handler.JRB2 = sim_obj.sim.getObject("/JRB2")
    obj_handler.JLF2 = sim_obj.sim.getObject("/JLF2")
    obj_handler.JLB2 = sim_obj.sim.getObject("/JLB2") 
    #Stores the different id for every handler starting with the base
    sim_obj.joint_handler_ids = list(handler.__dict__.values())

    sim_obj.num_joints = len(sim_obj.joint_handler_ids)

    sim_obj.obj_handler_ids = list(obj_handler.__dict__.values())

    sim_obj.sim.startSimulation()


def init_position():
    start_pos = ri(0,1)
    #Fixes the last joints position to 0 so they wont move
    for obj in sim_obj.obj_handler_ids[:-1]:
        sim_obj.sim.setJointTargetPosition(obj, 0)
    if start_pos == 0:
        #Set all the joints to their position
        for joint_init in sim_obj.joint_handler_ids:
            sim_obj.sim.setJointTargetPosition(joint_init, 0)
        #After setting all the joints it moves    
        for t_step_i in range(20):
            sim_obj.client.step()
    else:
        sim_obj.client.step()
    #Starting initial perceptions
    sim_obj.initial_j_positions = []
    sim_obj.initial_j_velocities = []
    sim_obj.initial_j_forces = []
    #Get initial perceptions
    base_x, base_y, base_z = sim_obj.sim.getObjectPosition(obj_handler.base, sim_obj.sim.handle_world)
    base_alpha, base_beta, base_gamma = sim_obj.sim.getObjectOrientation(obj_handler.base, sim_obj.sim.handle_world)
    sim_obj.initial_base_pos_x = base_x
    sim_obj.initial_base_pos_x = base_y
    sim_obj.initial_base_pos_x = base_z
    sim_obj.initial_base_ori_alpha = base_alpha
    sim_obj.initial_base_ori_beta = base_beta
    sim_obj.initial_base_ori_gamma = base_gamma    
    #Joint perceptions
    for joint_i_per in sim_obj.joint_handler_ids:
        #Angular/linear position of the joint
        pos = sim_obj.sim.getJointPosition(joint_i_per)
        #Torque/force being aplied by the joint
        force = sim_obj.sim.getJointForce(joint_i_per)
        #Angular/linear velocity of the joint's movement
        veloc = sim_obj.sim.getJointVelocity(joint_i_per)
        sim_obj.initial_j_positions.append(pos)
        sim_obj.initial_j_velocities.append(veloc)
        sim_obj.initial_j_forces.append(force)


#Generate list of random positions
def rand_gen():
    for num in range(sim_obj.num_joints):
        n = ru(-7.5, 7.5) * math.pi / 180
        perception.increments.append(n)


#Function to move all the joints to a random position
def move_joints():
    if perception.step != 0:
        for joint_num in range(sim_obj.num_joints):
            prev_pos = sim_obj.training_data[-1]['post_j_positions']
            sim_obj.sim.setJointTargetPosition(sim_obj.joint_handler_ids[joint_num], (perception.increments[joint_num]+prev_pos[joint_num]))
        #After setting all the joints it moves
        for t_step in range(5):
            sim_obj.client.step()
    else:
        for joint_num in range(sim_obj.num_joints):
            sim_obj.sim.setJointTargetPosition(sim_obj.joint_handler_ids[joint_num], (perception.increments[joint_num]+sim_obj.initial_j_positions[joint_num]))
        #After setting all the joints it moves
        for t_step in range(5):
            sim_obj.client.step()


def get_preceptions():
    #Define previous perceptions
    if perception.step != 0:
        perception.prev_base_pos_x = sim_obj.training_data[-1]['post_base_pos_x']
        perception.prev_base_pos_y = sim_obj.training_data[-1]['post_base_pos_y']
        perception.prev_base_pos_z = sim_obj.training_data[-1]['post_base_pos_z']
        perception.prev_base_ori_alpha = sim_obj.training_data[-1]['post_base_ori_alpha']
        perception.prev_base_ori_beta = sim_obj.training_data[-1]['post_base_ori_beta']
        perception.prev_base_ori_gamma = sim_obj.training_data[-1]['post_base_ori_gamma']
        perception.prev_j_positions = sim_obj.training_data[-1]['post_j_positions']
        perception.prev_j_velocities = sim_obj.training_data[-1]['post_j_velocities']
        perception.prev_j_forces = sim_obj.training_data[-1]['post_j_forces']    
    else:
        perception.prev_base_pos_x = sim_obj.initial_base_pos_x 
        perception.prev_base_pos_y = sim_obj.initial_base_pos_y
        perception.prev_base_pos_z = sim_obj.initial_base_pos_z
        perception.prev_base_ori_alpha = sim_obj.initial_base_ori_alpha
        perception.prev_base_ori_beta = sim_obj.initial_base_ori_beta
        perception.prev_base_ori_gamma = sim_obj.initial_base_ori_gamma
        perception.prev_j_positions = sim_obj.initial_j_positions
        perception.prev_j_velocities = sim_obj.initial_j_velocities
        perception.prev_j_forces = sim_obj.initial_j_forces 

    #Get perceptions
    base_x, base_y, base_z = sim_obj.sim.getObjectPosition(obj_handler.base, sim_obj.sim.handle_world)
    base_alpha, base_beta, base_gamma = sim_obj.sim.getObjectOrientation(obj_handler.base, sim_obj.sim.handle_world)
    perception.post_base_pos_x = base_x
    perception.post_base_pos_y = base_y
    perception.post_base_pos_z = base_z
    perception.post_base_ori_alpha = base_alpha
    perception.post_base_ori_beta = base_beta
    perception.post_base_ori_gamma = base_gamma
        
    #Joint perceptions
    for joint_per in sim_obj.joint_handler_ids:
        #Angular/linear position of the joint
        pos = sim_obj.sim.getJointPosition(joint_per)
        #Torque/force being aplied by the joint
        force = sim_obj.sim.getJointForce(joint_per)
        #Angular/linear velocity of the joint's movement
        veloc = sim_obj.sim.getJointVelocity(joint_per)
        perception.post_j_positions.append(pos)
        perception.post_j_velocities.append(veloc)
        perception.post_j_forces.append(force)


def export():    
    #Dump data in file
    timestr = time.strftime("_%Y%d%m_%H%M%S_")
    sim_obj.file = open('training_dataset_' + str(port_conexion) + timestr + '.pkl', 'wb')
    pickle.dump(sim_obj.training_data, sim_obj.file)
    sim_obj.file.close()
    if not cluster:
        df = pd.DataFrame(sim_obj.training_data)
        df.to_csv('data_' + str(port_conexion) + timestr + '.csv', index=False)



In [14]:

#Main function for multiple executions and multiple steps
#def main():
sim_obj = sim_objects()
handler = joint_handlers_class()
obj_handler = obj_handlers_class()
perception = perceptions()
for seq in range(sim_obj.sequencies):
    load_quad_class()
    init_position()
    for seq_step in range(sim_obj.seq_steps):
        perception = perceptions()
        perception.sequence = seq
        perception.step = seq_step    
        rand_gen()
        #Execute every joint action
        move_joints()
        get_preceptions()
        sim_obj.training_data.append(vars(perception))
    sim_obj.sim.stopSimulation()
#df = pd.DataFrame(sim_obj.training_data)
#df.to_csv('data_using_main.csv', index=False)
    #export()

In [11]:
if __name__ == "__main__":
    main()

NameError: name 'sim_obj' is not defined